You've started a new movie-rating website, and you've been collecting data on reviewers' ratings of various movies. There's not much data yet, but you can still try out some interesting queries. Here's the schema:

Movie ( mID, title, year, director )
English: There is a movie with ID number mID, a title, a release year, and a director.

Reviewer ( rID, name )
English: The reviewer with ID number rID has a certain name.

Rating ( rID, mID, stars, ratingDate )
English: The reviewer rID gave the movie mID a number of stars rating (1-5) on a certain ratingDate.

Your queries will run over a small data set conforming to the schema. View the database. (You can also download the schema and data.)

Instructions: Each problem asks you to write a query in SQL. 

In [2]:
# import libraries
import pandas as pd
import sqlite3

In [4]:
# connect to database
conn = sqlite3.connect(':memory:')

# create cursor object
cur = conn.cursor()

In [6]:
# read in csv files to dataframe and write records 
# stored in the DataFrame to a SQL database

df1 = pd.read_csv('movie.csv', )
df1.to_sql('movie', con = conn, index=False)
df2 = pd.read_csv('rating.csv', )
df2.to_sql('rating', con = conn, index=False)
df3 = pd.read_csv('reviewer.csv', )
df3.to_sql('reviewer', con = conn, index=False)

In [7]:
# view movie dataframe

cur.execute('''SELECT * FROM movie;''')            # sql query
movie = pd.DataFrame(cur.fetchall())               # create dataframe from sql query
movie.columns = [x[0] for x in cur.description]    # labels dataframe columns
movie                                              # view dataframe

,mID,title,year,director
0,101,Gone with the Wind,1939,Victor Fleming
1,102,Star Wars,1977,George Lucas
2,103,The Sound of Music,1965,Robert Wise
3,104,E.T.,1982,Steven Spielberg
4,105,Titanic,1997,James Cameron
5,106,Snow White,1937,None
6,107,Avatar,2009,James Cameron
7,108,Raiders of the Lost Ark,1981,Steven Spielberg


In [8]:
# view rating dataframe

cur.execute('''SELECT * FROM rating;''')            # sql query
rating = pd.DataFrame(cur.fetchall())               # create dataframe from sql query
rating.columns = [x[0] for x in cur.description]    # labels dataframe columns
rating                                              # view dataframe

,rID,mID,stars,ratingDate
0,201,101,2,2011-01-22
1,201,101,4,2011-01-27
2,202,106,4,None
3,203,103,2,2011-01-20
4,203,108,4,2011-01-12
5,203,108,2,2011-01-30
6,204,101,3,2011-01-09
7,205,103,3,2011-01-27
8,205,104,2,2011-01-22
9,205,108,4,None


In [9]:
# view reviewer dataframe

cur.execute('''SELECT * FROM reviewer;''')            # sql query
reviewer = pd.DataFrame(cur.fetchall())               # create dataframe from sql query
reviewer.columns = [x[0] for x in cur.description]    # labels dataframe columns
reviewer                                              # view dataframe

,rID,name
0,201,Sarah Martinez
1,202,Daniel Lewis
2,203,Brittany Harris
3,204,Mike Anderson
4,205,Chris Jackson
5,206,Elizabeth Thomas
6,207,James Cameron
7,208,Ashley White


In [ ]:
# 1. Find the titles of all movies directed by Steven Spielberg.

SELECT title
FROM movie
WHERE director = 'Steven Spielberg';

In [ ]:
# 2. Find all years that have a movie that received a rating of 4 or 5, and sort them in increasing order.

SELECT DISTINCT m.year
FROM movie m
    JOIN rating r
        ON m.mID = r.mID
WHERE r.stars IN (4, 5)
ORDER BY year;

In [ ]:
# 3. Find the titles of all movies that have no ratings.

SELECT DISTINCT m.title
FROM movie m
    LEFT JOIN rating r
        ON m.mID = r.mID
WHERE r.stars IS NULL;

In [ ]:
# 4. Some reviewers didn't provide a date 
# with their rating. Find the names of 
# all reviewers who have ratings with a 
# NULL value for the date.

SELECT rev.name
FROM rating r
    JOIN reviewer rev
        ON r.rID = rev.rID
WHERE r.ratingDate IS NULL;

In [ ]:
# 5. Write a query to return the ratings 
# data in a more readable format: 
# reviewer name, movie title, stars, 
# and ratingDate. Also, sort the data, 
# first by reviewer name, then by movie 
# title, and lastly by number of stars.

SELECT rev.name, m.title, r.stars, r.ratingDate
FROM movie m
    JOIN rating r
        ON m.mID = r.mID
            JOIN reviewer rev
                ON r.rID = rev.rID
ORDER BY 1, 2, 3;

In [ ]:
# 6. For all cases where the same reviewer 
# rated the same movie twice and gave it 
# a higher rating the second time, return the 
# reviewer's name and the title of the movie.

SELECT rev.name, m.title
FROM movie m
JOIN rating r1 ON m.mID = r1.mID
JOIN rating r2 ON r1.rID = r2.rID
JOIN reviewer rev ON r2.rID = rev.rID
WHERE r1.mID = r2.mID AND r1.ratingDate > r2.ratingDate AND r1.stars > r2.stars;

In [ ]:
# 7. For each movie that has at least one rating, 
# find the highest number of stars that movie 
# received. Return the movie title and number 
# of stars. Sort by movie title.

SELECT m.title, MAX(r.stars)
FROM movie m 
JOIN rating r
ON m.mID = r.mID
WHERE r.stars >= 1
GROUP BY m.title
ORDER BY m.title;  

In [ ]:
# 8. For each movie, return the title and 
# the 'rating spread', that is, the 
# difference between highest and lowest 
# ratings given to that movie. Sort by 
# rating spread from highest to lowest, 
# then by movie title.

SELECT m.title, MAX(r.stars) - MIN(r.stars) AS "Rating Spread"
FROM movie m
JOIN rating r
ON m.mID = r.mID
GROUP BY m.title
ORDER BY 2 DESC, 1;

In [ ]:
# 9. Find the difference between the average 
# rating of movies released before 1980 
# and the average rating of movies released 
# after 1980. (Make sure to calculate the 
# average rating for each movie, then the 
# average of those averages for movies before 
# 1980 and movies after. Don't just calculate 
# the overall average rating before and after 1980.)

SELECT AVG(before1980.avg) - AVG(after1980.avg) AS difference
FROM 
(SELECT AVG(r.stars) AS avg
FROM movie m
JOIN rating r
ON m.mID = r.mID
WHERE year < 1980
GROUP BY m.title) AS before1980,
(SELECT AVG(r.stars) AS avg
FROM movie m
JOIN rating r
ON m.mID = r.mID
WHERE year > 1980
GROUP BY m.title) AS after1980;